In [0]:
# Dependencies to load moduels from this repo
import importlib.util
import sys

# Load cv module directly from file path
cv_path = "/Workspace/Shared/Team 4_2/flight-departure-delay-predictive-modeling/notebooks/Cross Validator/cv.py"
spec = importlib.util.spec_from_file_location("cv", cv_path)
cv = importlib.util.module_from_spec(spec)
spec.loader.exec_module(cv)

# Dependencies for time series features
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_timestamp, date_format
import pandas as pd
import numpy as np
from prophet import Prophet

# Dependencies for EDA
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss, coint
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Other Dependencies
import time

# Path for persistent storage
FOLDER_PATH = "dbfs:/mnt/mids-w261/student-groups/Group_4_2/experiments"

## Load Training Data

In [0]:
# Load from data_loader and save snapshot (run once)
ts_data_path = f"{FOLDER_PATH}/timeseries_data_snapshot.parquet"

print("Loading from data_loader and saving snapshot...")
start = time.time()
data_loader = cv.FlightDelayDataLoader()
data_loader.load()
folds = data_loader.get_version("12M")

# Use final fold (test fold) which has 4 years of data for time series analysis
# This allows us to learn yearly seasonality without data leakage
train_df, test_df = folds[-1]
ts_data = train_df

# Check partition count and repartition if needed
num_partitions = ts_data.rdd.getNumPartitions()
if num_partitions > 500:
    ts_data = ts_data.coalesce(200)
elif num_partitions < 10:
    ts_data = ts_data.repartition(50)

# Save snapshot
ts_data.write.mode("overwrite").parquet(ts_data_path)
print(f"Saved snapshot in {time.time() - start:.2f} seconds")

print(f"\nTime series data: {ts_data.count():,} flights")
print(f"Date range: {ts_data.agg(F.min('FL_DATE'), F.max('FL_DATE')).collect()}")

In [0]:
# Load from saved snapshot (run this on subsequent runs, skip above cell
ts_data_path = f"{FOLDER_PATH}/timeseries_data_snapshot.parquet"

print(f"Loading timeseries data from {ts_data_path}...")
start = time.time()
ts_data = spark.read.parquet(ts_data_path)
ts_data.count()  # Materialize
print(f"Loaded in {time.time() - start:.2f} seconds")

print(f"\nTime series data: {ts_data.count():,} flights")
print(f"Date range: {ts_data.agg(F.min('FL_DATE'), F.max('FL_DATE')).collect()}")

## Generate Time-Series

In [0]:
# Prepare date column for aggregation
# Convert FL_DATE to date type and filter valid data
ts_data_prep = ts_data.withColumn(
    "date", 
    to_timestamp(col("FL_DATE"), "yyyy-MM-dd").cast("date")
).filter(
    col("date").isNotNull() & 
    col("DEP_DELAY").isNotNull()
)

### Global Time-Series

In [0]:
# lobal time series: Average departure delay by date
global_dep_delays_spark = (
    ts_data_prep
    .groupBy("date")
    .agg(
        F.avg("DEP_DELAY").alias("avg_dep_delay"),
        F.count("*").alias("flight_count")
    )
    .orderBy("date")
)

# Convert to pandas for Prophet
global_dep_delays = global_dep_delays_spark.toPandas()
global_dep_delays['ds'] = pd.to_datetime(global_dep_delays['date'])
global_dep_delays = global_dep_delays.rename(columns={'avg_dep_delay': 'y'})

print("Global time series (first 10 days):")
print(global_dep_delays[['ds', 'y', 'flight_count']].head(10))
print(f"\nTotal days: {len(global_dep_delays)}")


### Per Airport Time-Series

In [0]:
# Per-airport time series: Departure delays, arrival delays, and flight counts
# Aggregate by airport and date
per_airport_ts_spark = (
    ts_data_prep
    .groupBy("origin", "date")
    .agg(
        F.avg("DEP_DELAY").alias("avg_dep_delay"),
        F.avg("ARR_DELAY").alias("avg_arr_delay"),
        F.count("*").alias("flight_count")
    )
    .orderBy("origin", "date")
)

# Convert to pandas
per_airport_ts = per_airport_ts_spark.toPandas()
per_airport_ts['ds'] = pd.to_datetime(per_airport_ts['date'])

print(f"Per-airport time series: {len(per_airport_ts):,} rows")
print(f"Number of airports: {per_airport_ts['origin'].nunique()}")
print(f"Average days per airport: {len(per_airport_ts) / per_airport_ts['origin'].nunique():.1f}")
print("\nSample (first airport):")
first_airport = per_airport_ts['origin'].iloc[0]
print(per_airport_ts[per_airport_ts['origin'] == first_airport][['origin', 'ds', 'avg_dep_delay', 'avg_arr_delay', 'flight_count']].head(10))


## Time-Series EDA


In [0]:
import matplotlib.pyplot as plt
### 1. Raw Time Series Plots


# Global departure delays
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Full time series
axes[0].plot(global_dep_delays['ds'], global_dep_delays['y'], linewidth=0.5, alpha=0.7)
axes[0].set_title('Global Average Departure Delay Over Time (Full Series)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Average Departure Delay (minutes)')
axes[0].grid(True, alpha=0.3)

# Zoomed view (last 6 months)
last_6mo = global_dep_delays.tail(180)
axes[1].plot(last_6mo['ds'], last_6mo['y'], linewidth=1, marker='o', markersize=2)
axes[1].set_title('Global Average Departure Delay (Last 6 Months)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Average Departure Delay (minutes)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print("\n=== Global Departure Delay Summary Statistics ===")
print(global_dep_delays['y'].describe())


In [0]:
### 2. Distribution and Box Plots

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histogram
axes[0, 0].hist(global_dep_delays['y'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribution of Daily Average Departure Delays', fontweight='bold')
axes[0, 0].set_xlabel('Average Departure Delay (minutes)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(global_dep_delays['y'].mean(), color='r', linestyle='--', label=f'Mean: {global_dep_delays["y"].mean():.2f}')
axes[0, 0].legend()

# Box plot by year
global_dep_delays['year'] = global_dep_delays['ds'].dt.year
sns.boxplot(data=global_dep_delays, x='year', y='y', ax=axes[0, 1])
axes[0, 1].set_title('Departure Delays by Year', fontweight='bold')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Average Departure Delay (minutes)')

# Box plot by month
global_dep_delays['month'] = global_dep_delays['ds'].dt.month
sns.boxplot(data=global_dep_delays, x='month', y='y', ax=axes[1, 0])
axes[1, 0].set_title('Departure Delays by Month', fontweight='bold')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Average Departure Delay (minutes)')

# Box plot by day of week
global_dep_delays['day_of_week'] = global_dep_delays['ds'].dt.dayofweek
sns.boxplot(data=global_dep_delays, x='day_of_week', y='y', ax=axes[1, 1])
axes[1, 1].set_title('Departure Delays by Day of Week', fontweight='bold')
axes[1, 1].set_xlabel('Day of Week (0=Monday)')
axes[1, 1].set_ylabel('Average Departure Delay (minutes)')

plt.tight_layout()
plt.show()

In [0]:
### 3. Stationarity Tests

# Set up time series for testing (remove NaN values)

ts_values = global_dep_delays['y'].dropna().values

print("=== Stationarity Tests ===\n")

# Augmented Dickey-Fuller Test (ADF)
print("1. Augmented Dickey-Fuller Test (ADF):")
print("   H0: Series has a unit root (non-stationary)")
print("   H1: Series is stationary\n")
adf_result = adfuller(ts_values)
print(f"   ADF Statistic: {adf_result[0]:.4f}")
print(f"   p-value: {adf_result[1]:.4f}")
print(f"   Critical Values:")
for key, value in adf_result[4].items():
    print(f"      {key}: {value:.4f}")
if adf_result[1] <= 0.05:
    print("   ✓ Series is STATIONARY (reject H0, p < 0.05)")
else:
    print("   ✗ Series is NON-STATIONARY (fail to reject H0, p >= 0.05)")

print("\n" + "="*60 + "\n")

# KPSS Test
print("2. KPSS Test:")
print("   H0: Series is stationary")
print("   H1: Series has a unit root (non-stationary)\n")
kpss_result = kpss(ts_values, regression='ct')  # 'ct' = constant and trend
print(f"   KPSS Statistic: {kpss_result[0]:.4f}")
print(f"   p-value: {kpss_result[1]:.4f}")
print(f"   Critical Values:")
for key, value in kpss_result[3].items():
    print(f"      {key}: {value:.4f}")
if kpss_result[1] >= 0.05:
    print("   ✓ Series is STATIONARY (fail to reject H0, p >= 0.05)")
else:
    print("   ✗ Series is NON-STATIONARY (reject H0, p < 0.05)")

In [0]:
### 4. Autocorrelation Analysis

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# ACF (Autocorrelation Function)
plot_acf(global_dep_delays['y'].dropna(), lags=50, ax=axes[0], alpha=0.05)
axes[0].set_title('Autocorrelation Function (ACF)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Lag')
axes[0].set_ylabel('Autocorrelation')

# PACF (Partial Autocorrelation Function)
plot_pacf(global_dep_delays['y'].dropna(), lags=50, ax=axes[1], alpha=0.05)
axes[1].set_title('Partial Autocorrelation Function (PACF)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel('Partial Autocorrelation')

plt.tight_layout()
plt.show()

# Ljung-Box Test for autocorrelation
print("\n=== Ljung-Box Test for Autocorrelation ===")
print("H0: No autocorrelation")
print("H1: Autocorrelation exists\n")
lb_result = acorr_ljungbox(global_dep_delays['y'].dropna(), lags=10, return_df=True)
print(lb_result)
if (lb_result['lb_pvalue'] < 0.05).any():
    print("\n✗ Significant autocorrelation detected (p < 0.05)")
else:
    print("\n✓ No significant autocorrelation (p >= 0.05)")

In [0]:
### 5. STL Decomposition (Seasonal and Trend decomposition using Loess)

# Set date as index for STL
ts_indexed = global_dep_delays.set_index('ds')['y'].dropna()

# STL Decomposition
# period=365 for yearly seasonality, but we can also try weekly (period=7)
stl = STL(ts_indexed, seasonal=365, trend=None, robust=True)
decomposition = stl.fit()

fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Original
axes[0].plot(ts_indexed.index, ts_indexed.values, linewidth=0.5, alpha=0.7)
axes[0].set_title('Original Time Series', fontweight='bold')
axes[0].set_ylabel('Departure Delay')
axes[0].grid(True, alpha=0.3)

# Trend
axes[1].plot(decomposition.trend.index, decomposition.trend.values, linewidth=1, color='blue')
axes[1].set_title('Trend Component', fontweight='bold')
axes[1].set_ylabel('Trend')
axes[1].grid(True, alpha=0.3)

# Seasonal
axes[2].plot(decomposition.seasonal.index, decomposition.seasonal.values, linewidth=0.5, alpha=0.7, color='green')
axes[2].set_title('Seasonal Component', fontweight='bold')
axes[2].set_ylabel('Seasonal')
axes[2].grid(True, alpha=0.3)

# Residual
axes[3].plot(decomposition.resid.index, decomposition.resid.values, linewidth=0.5, alpha=0.7, color='red')
axes[3].set_title('Residual Component', fontweight='bold')
axes[3].set_ylabel('Residual')
axes[3].set_xlabel('Date')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary of decomposition
print("\n=== STL Decomposition Summary ===")
print(f"Trend variance: {decomposition.trend.var():.4f}")
print(f"Seasonal variance: {decomposition.seasonal.var():.4f}")
print(f"Residual variance: {decomposition.resid.var():.4f}")
print(f"\nResidual statistics:")
print(decomposition.resid.describe())

In [0]:
### 6. Additional Statistical Analysis

# Monthly averages
monthly_avg = global_dep_delays.groupby('month')['y'].mean()
print("=== Monthly Average Departure Delays ===")
for month, avg in monthly_avg.items():
    month_name = pd.Timestamp(2020, month, 1).strftime('%B')
    print(f"{month_name:12s}: {avg:6.2f} minutes")

# Day of week averages
dow_avg = global_dep_delays.groupby('day_of_week')['y'].mean()
dow_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
print("\n=== Day of Week Average Departure Delays ===")
for dow, avg in dow_avg.items():
    print(f"{dow_names[dow]:12s}: {avg:6.2f} minutes")

# Year-over-year comparison
print("\n=== Year-over-Year Comparison ===")
yearly_avg = global_dep_delays.groupby('year')['y'].agg(['mean', 'std', 'min', 'max'])
print(yearly_avg)

**TODO**: Additional Time Series Analysis

- [ ] **Delays by Airplane Model**: Time series of average departure delays by aircraft model (e.g., Boeing 737, Airbus A320)
- [ ] **Delays by Carrier**: Time series of average departure delays by airline carrier
- [ ] Apply same EDA (stationarity tests, STL decomposition, etc.) to these additional time series

In [0]:
### 7. Cointegration Test: Flight Count vs Average Delay

# Test if number of flights and average delay are cointegrated
# Cointegration means they have a long-term equilibrium relationship
# even if individually non-stationary

# Align the series (same dates)
flight_count_series = global_dep_delays['flight_count'].dropna().values
delay_series = global_dep_delays['y'].dropna().values

# Ensure same length
min_len = min(len(flight_count_series), len(delay_series))
flight_count_series = flight_count_series[:min_len]
delay_series = delay_series[:min_len]

print("=== Cointegration Test: Flight Count vs Average Delay ===\n")
print("H0: No cointegration (series are not in long-term equilibrium)")
print("H1: Cointegration exists (series have long-term relationship)\n")

# Engle-Granger cointegration test
coint_result = coint(delay_series, flight_count_series)

print(f"Cointegration Test Statistic: {coint_result[0]:.4f}")
print(f"p-value: {coint_result[1]:.4f}")
print(f"Critical Values:")
for key, value in coint_result[2].items():
    print(f"   {key}: {value:.4f}")

if coint_result[1] <= 0.05:
    print("\n✓ COINTEGRATION DETECTED (p < 0.05)")
    print("  → Flight count and delay have a long-term equilibrium relationship")
    print("  → They move together over time despite short-term deviations")
else:
    print("\n✗ No cointegration (p >= 0.05)")
    print("  → Flight count and delay do not have a stable long-term relationship")

# Visualize the relationship
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot both series
ax1 = axes[0]
ax1_twin = ax1.twinx()
ax1.plot(global_dep_delays['ds'].iloc[:min_len], delay_series, 'b-', label='Avg Delay', linewidth=1, alpha=0.7)
ax1_twin.plot(global_dep_delays['ds'].iloc[:min_len], flight_count_series, 'r-', label='Flight Count', linewidth=1, alpha=0.7)
ax1.set_xlabel('Date')
ax1.set_ylabel('Average Delay (minutes)', color='b')
ax1_twin.set_ylabel('Flight Count', color='r')
ax1.set_title('Flight Count vs Average Delay Over Time', fontweight='bold')
ax1.tick_params(axis='y', labelcolor='b')
ax1_twin.tick_params(axis='y', labelcolor='r')
ax1.grid(True, alpha=0.3)

# Scatter plot
axes[1].scatter(flight_count_series, delay_series, alpha=0.3, s=10)
axes[1].set_xlabel('Flight Count')
axes[1].set_ylabel('Average Departure Delay (minutes)')
axes[1].set_title('Flight Count vs Average Delay (Scatter)', fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Add correlation
correlation = np.corrcoef(flight_count_series, delay_series)[0, 1]
axes[1].text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
             transform=axes[1].transAxes, fontsize=12,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

## Prophet Feature Generation

Use Prophet to extract time-series features (trend, seasonality, forecasts) that can be used as features in ML models.

In [0]:
# Fit Prophet model on global time series
print("Fitting Prophet model on global departure delays...")

# Prepare data for Prophet (requires 'ds' and 'y' columns)
prophet_data = global_dep_delays[['ds', 'y']].copy()
prophet_data = prophet_data.dropna()

# Initialize and fit Prophet model
# Enable yearly and weekly seasonality
prophet_model_global = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,  # Daily doesn't make sense for daily aggregated data
    seasonality_mode='multiplicative',  # or 'additive'
    interval_width=0.95,  # 95% confidence intervals
    changepoint_prior_scale=0.05  # Controls flexibility of trend changes
)

prophet_model_global.fit(prophet_data)

# Generate forecast for all dates in the training data (and potentially future dates)
# This gives us trend, seasonality components, and forecast values
forecast_global = prophet_model_global.predict(prophet_data[['ds']])

print(f"Prophet forecast generated for {len(forecast_global)} dates")
print("\nForecast columns:")
print(forecast_global.columns.tolist())
print("\nSample forecast:")
print(forecast_global[['ds', 'yhat', 'trend', 'yearly', 'weekly']].head(10))

In [0]:
# Extract Prophet features for joining back to flight data
# These features can be used in ML models

prophet_features_global = forecast_global[[
    'ds',
    'trend',              # Long-term trend component
    'yearly',             # Yearly seasonality component
    'weekly',             # Weekly seasonality component
    'yhat',               # Forecasted value (trend + seasonality)
    'yhat_lower',         # Lower bound of forecast interval
    'yhat_upper',         # Upper bound of forecast interval
]].copy()

# Rename for clarity when joining
prophet_features_global = prophet_features_global.rename(columns={
    'trend': 'prophet_trend_global',
    'yearly': 'prophet_yearly_seasonality_global',
    'weekly': 'prophet_weekly_seasonality_global',
    'yhat': 'prophet_forecast_global',
    'yhat_lower': 'prophet_forecast_lower_global',
    'yhat_upper': 'prophet_forecast_upper_global'
})

# Convert date to string format matching FL_DATE
prophet_features_global['date_str'] = prophet_features_global['ds'].dt.strftime('%Y-%m-%d')

print("Global Prophet features (sample):")
print(prophet_features_global.head(10))
print(f"\nTotal feature rows: {len(prophet_features_global)}")


### Per-Airport Prophet Models

In [0]:
# Fit Prophet models for each airport
# This will take longer but provides airport-specific trend and seasonality features

print("Fitting Prophet models for each airport...")
print(f"Total airports: {per_airport_ts['origin'].nunique()}")

# Check data availability per airport
airport_day_counts = per_airport_ts.groupby('origin')['ds'].count().sort_values(ascending=False)
print(f"\nData availability statistics:")
print(f"  Min days: {airport_day_counts.min()}")
print(f"  Max days: {airport_day_counts.max()}")
print(f"  Mean days: {airport_day_counts.mean():.1f}")
print(f"  Median days: {airport_day_counts.median():.1f}")

# Use a lower threshold - need at least 14 days for weekly seasonality
# With only 3 quarters (~270 days), we can't do yearly seasonality, but we can get trend and weekly patterns
min_days_required = 14  # At least 2 weeks for weekly seasonality
airports_with_sufficient_data = airport_day_counts[airport_day_counts >= min_days_required].index.tolist()

print(f"\nAirports with >= {min_days_required} days of data: {len(airports_with_sufficient_data)}")

# Fit Prophet for a subset of airports first (for testing)
# In production, fit for all airports
top_airports = airports_with_sufficient_data[:20]  # Start with top 20 airports
print(f"\nFitting Prophet models for {len(top_airports)} airports (sample)...")

prophet_features_per_airport = []

for airport in top_airports:
    airport_data = per_airport_ts[per_airport_ts['origin'] == airport].sort_values('ds')
    airport_prophet_data = airport_data[['ds', 'avg_dep_delay']].copy()
    airport_prophet_data = airport_prophet_data.rename(columns={'avg_dep_delay': 'y'})
    airport_prophet_data = airport_prophet_data.dropna()
    
    if len(airport_prophet_data) < min_days_required:
        continue
    
    try:
        # Determine seasonality based on data availability
        has_enough_for_yearly = len(airport_prophet_data) >= 365
        has_enough_for_weekly = len(airport_prophet_data) >= 14
        
        # Fit Prophet model with appropriate seasonality
        prophet_model = Prophet(
            yearly_seasonality=has_enough_for_yearly,  # Only if >=365 days
            weekly_seasonality=has_enough_for_weekly,  # Only if >=14 days
            daily_seasonality=False,
            seasonality_mode='multiplicative',
            interval_width=0.95,
            changepoint_prior_scale=0.05
        )
        prophet_model.fit(airport_prophet_data)
        
        # Generate forecast
        forecast = prophet_model.predict(airport_prophet_data[['ds']])
        
        # Extract features (yearly may not exist if not enough data)
        feature_cols = ['ds', 'trend', 'weekly', 'yhat', 'yhat_lower', 'yhat_upper']
        if 'yearly' in forecast.columns:
            feature_cols.insert(2, 'yearly')  # Insert after 'trend'
        
        features = forecast[feature_cols].copy()
        features['origin'] = airport
        
        # Rename columns
        rename_dict = {
            'trend': 'prophet_trend_origin',
            'weekly': 'prophet_weekly_seasonality_origin',
            'yhat': 'prophet_forecast_origin',
            'yhat_lower': 'prophet_forecast_lower_origin',
            'yhat_upper': 'prophet_forecast_upper_origin'
        }
        if 'yearly' in features.columns:
            rename_dict['yearly'] = 'prophet_yearly_seasonality_origin'
        
        features = features.rename(columns=rename_dict)
        features['date_str'] = features['ds'].dt.strftime('%Y-%m-%d')
        
        prophet_features_per_airport.append(features)
        
    except Exception as e:
        print(f"Error fitting Prophet for airport {airport}: {str(e)}")
        continue

# Combine all airport features
if prophet_features_per_airport:
    prophet_features_airport_df = pd.concat(prophet_features_per_airport, ignore_index=True)
    print(f"\n✓ Generated Prophet features for {prophet_features_airport_df['origin'].nunique()} airports")
    print(f"Total feature rows: {len(prophet_features_airport_df)}")
    print("\nSample features:")
    print(prophet_features_airport_df.head(10))
else:
    print("\nNo Prophet features generated for airports")
